In [1]:
import tensorflow as tf
from tensorflow.keras import layers
print(tf.__version__)
print(tf.keras.__version__)

C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passi

1.14.0
2.2.4-tf


In [2]:

model = tf.keras.Sequential()
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))
model.compile(optimizer=tf.keras.optimizers.Adam(0.001),
             loss=tf.keras.losses.categorical_crossentropy,
             metrics=[tf.keras.metrics.categorical_accuracy])

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [3]:
import numpy as np

train_x = np.random.random((1000, 72))
train_y = np.random.random((1000, 10))

val_x = np.random.random((200, 72))
val_y = np.random.random((200, 10))

model.fit(train_x, train_y, epochs=10, batch_size=100,
          validation_data=(val_x, val_y))

Train on 1000 samples, validate on 200 samples
Epoch 1/10
1000/1000 [==============================] - 0s 185us/sample - loss: 12.0453 - categorical_accuracy: 0.1030 - val_loss: 12.3141 - val_categorical_accuracy: 0.1050
Epoch 2/10
1000/1000 [==============================] - 0s 19us/sample - loss: 12.3095 - categorical_accuracy: 0.1030 - val_loss: 12.8122 - val_categorical_accuracy: 0.1050
Epoch 3/10
1000/1000 [==============================] - 0s 33us/sample - loss: 13.0001 - categorical_accuracy: 0.1030 - val_loss: 13.8022 - val_categorical_accuracy: 0.1050
Epoch 4/10
1000/1000 [==============================] - 0s 17us/sample - loss: 14.3241 - categorical_accuracy: 0.1020 - val_loss: 15.6822 - val_categorical_accuracy: 0.1100
Epoch 5/10
1000/1000 [==============================] - 0s 18us/sample - loss: 16.8091 - categorical_accuracy: 0.

In [4]:
dataset = tf.data.Dataset.from_tensor_slices((train_x, train_y))
dataset = dataset.batch(32)
dataset = dataset.repeat()
val_dataset = tf.data.Dataset.from_tensor_slices((val_x, val_y))
val_dataset = val_dataset.batch(32)
val_dataset = val_dataset.repeat()

model.fit(dataset, epochs=10, steps_per_epoch=30,
          validation_data=val_dataset, validation_steps=3)

Epoch 1/10
30/30 [==============================] - 0s 6ms/step - loss: 85.8313 - categorical_accuracy: 0.0979 - val_loss: 120.2701 - val_categorical_accuracy: 0.1042
Epoch 2/10
30/30 [==============================] - 0s 1ms/step - loss: 153.5273 - categorical_accuracy: 0.1004 - val_loss: 201.5929 - val_categorical_accuracy: 0.1146
Epoch 3/10
30/30 [==============================] - 0s 1ms/step - loss: 245.8081 - categorical_accuracy: 0.1079 - val_loss: 308.1039 - val_categorical_accuracy: 0.1042
Epoch 4/10
30/30 [==============================] - 0s 1ms/step - loss: 359.2890 - categorical_accuracy: 0.1132 - val_loss: 434.5970 - val_categorical_accuracy: 0.0833
Epoch 5/10
30/30 [==============================] - 0s 1ms/step - loss: 489.2606 - categorical_accuracy: 0.0951 - val_loss: 576.9846 - val_categorical_accuracy: 0.1354

In [5]:

test_x = np.random.random((1000, 72))
test_y = np.random.random((1000, 10))
model.evaluate(test_x, test_y, batch_size=32)
test_data = tf.data.Dataset.from_tensor_slices((test_x, test_y))
test_data = test_data.batch(32).repeat()
model.evaluate(test_data, steps=30)
# predict
result = model.predict(test_x, batch_size=32)
print(result)

30/30 [==============================] - 0s 3ms/step - loss: 1107.6290 - categorical_accuracy: 0.1167
[[0.         0.         0.         ... 0.4956868  0.         0.07349761]
 [0.         0.         0.         ... 0.5714214  0.         0.08382182]
 [0.         0.         0.         ... 0.5212572  0.         0.05411476]
 ...
 [0.         0.         0.         ... 0.40741688 0.         0.09882326]
 [0.         0.         0.         ... 0.47528768 0.         0.10845998]
 [0.         0.         0.         ... 0.52968585 0.         0.07130119]]


In [6]:
class MyLayer(layers.Layer):
    def __init__(self, output_dim, **kwargs):
        self.output_dim = output_dim
        super(MyLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        shape = tf.TensorShape((input_shape[1], self.output_dim))
        self.kernel = self.add_weight(name='kernel1', shape=shape,
                                   initializer='uniform', trainable=True)
        super(MyLayer, self).build(input_shape)

    def call(self, inputs):
        return tf.matmul(inputs, self.kernel)

    def compute_output_shape(self, input_shape):
        shape = tf.TensorShape(input_shape).as_list()
        shape[-1] = self.output_dim
        return tf.TensorShape(shape)

    def get_config(self):
        base_config = super(MyLayer, self).get_config()
        base_config['output_dim'] = self.output_dim
        return base_config

    @classmethod
    def from_config(cls, config):
        return cls(**config)

model = tf.keras.Sequential(
[
    MyLayer(10),
    layers.Activation('softmax')
])


model.compile(optimizer=tf.keras.optimizers.RMSprop(0.001),
             loss=tf.keras.losses.categorical_crossentropy,
             metrics=['accuracy'])

model.fit(train_x, train_y, batch_size=16, epochs=5)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Epoch 1/5
1000/1000 [==============================] - 0s 94us/sample - loss: 11.6368 - acc: 0.1040
Epoch 2/5
1000/1000 [==============================] - 0s 40us/sample - loss: 11.6373 - acc: 0.1070
Epoch 3/5
1000/1000 [==============================] - 0s 44us/sample - loss: 11.6363 - acc: 0.1040
Epoch 4/5
1000/1000 [==============================] - 0s 52us/sample - loss: 11.6371 - acc: 0.0970
Epoch 5/5
1000/1000 [==============================] - 0s 72us/sample - loss: 11.6353 - acc: 0.0930

In [7]:
callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=2, monitor='val_loss'),
    tf.keras.callbacks.TensorBoard(log_dir='./logs')
]
model.fit(train_x, train_y, batch_size=16, epochs=5,
         callbacks=callbacks, validation_data=(val_x, val_y))

Train on 1000 samples, validate on 200 samples
Epoch 1/5
  16/1000 [..............................] - ETA: 0s - loss: 12.4324 - acc: 0.0625

NotFoundError: Failed to create a directory: ./logs\plugins\profile\2021-03-22_16-40-38; No such file or directory